<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/LLM_GPT2_QA_Finetune_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import transformers
print(transformers.__version__)

4.47.1


Zeros-shot prediction

In [ ]:
# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.pad_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

Q: What is the capital of France?
A: The capital of France


Finetuning - Instruct<br>
target_text = f"{example['answer']} {tokenizer.eos_token}"

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token

# Prepare 5 QA samples for instruction tuning
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']} Answer:"
    target_text = example["answer"]


    # Tokenize inputs and labels
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)
    labels = tokenizer(target_text, truncation=True, padding="max_length", max_length=256)

    # Set labels to ignore padding tokens
    inputs["labels"] = [label if label != tokenizer.pad_token_id else -100 for label in labels["input_ids"]]
    return inputs

# Convert samples to Hugging Face dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples)
tokenized_dataset_train = dataset_train.map(preprocess_data)

dataset_valid = Dataset.from_list(valid_qa_samples)
tokenized_dataset_valid = dataset_valid.map(preprocess_data)

# Calculate save_steps based on dataset size and batch size
batch_size = 5  # Define the batch size used for training
num_train_samples = len(tokenized_dataset_train)
save_steps = num_train_samples // batch_size  # Number of steps per epoch

training_args = TrainingArguments(
    output_dir="./gpt2-qa-finetuned",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save checkpoints based on number of steps
    save_steps=save_steps,  # Save model at the end of every epoch
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
    num_train_epochs=30,
    save_total_limit=10,
    push_to_hub=False,
    report_to=[],
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_safetensors=False,
)


# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_valid,
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Save the fine-tuned model
# model.save_pretrained("./gpt2-qa-finetuned")
trainer.save_model("./gpt2-qa-finetuned")
tokenizer.save_pretrained("./gpt2-qa-finetuned")

# Load the fine-tuned model for inference
model_name = "./gpt2-qa-finetuned"
model = GPT2LMHeadModel.from_pretrained(model_name).to("cuda")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.pad_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-43-89e1ce3bd04b>:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,11.019800,11.029708
2,10.843100,11.029708
3,10.374700,11.029708
4,10.995600,11.029708
5,10.864800,9.971808
6,9.661200,9.971808
7,9.606000,9.302272
8,8.682900,8.801450
9,8.158200,8.394403
10,7.584000,8.043612


Q: What is the capital of France?
A: Paris. capital. of. France. per cent. of. per cent. of. per cent. of. per cent. of. per cent. of. per cent. of. per cent. of. per cent. of. per cent


In [ ]:
# Save the fine-tuned model
model_name = "/content/gpt2-qa-finetuned/checkpoint-30"

# Load the fine-tuned model for inference
model = GPT2LMHeadModel.from_pretrained(model_name).to("cuda")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=10, pad_token_id=tokenizer.pad_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

Q: What is the capital of France?
A: Paris. capital. of. France. per cent


Finetuning - Instruct with train-valid scratch

In [3]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
import torch.nn.functional as F
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare 5 QA samples for training and validation
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']} Answer:"
    target_text = example["answer"]

    # Tokenize inputs and labels
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)
    labels = tokenizer(target_text, truncation=True, padding="max_length", max_length=256)

    # Set labels to ignore padding tokens
    inputs["labels"] = [label if label != tokenizer.pad_token_id else -100 for label in labels["input_ids"]]
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Convert to PyTorch DataLoader
batch_size = 2

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["labels"] for item in batch])
    return input_ids, attention_mask, labels

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Cross-entropy loss function ignoring padding tokens
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model based on validation loss
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./gpt2-qa-best-loss"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training and evaluation functions
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # Save best model based on validation loss
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10)

# Load the best fine-tuned model for inference
model_name = "./gpt2-qa-best-loss"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.pad_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Training Loss: 10.1111, Validation Loss: 9.2618
Best model saved at epoch 1 with validation loss: 9.2618
Epoch 2/10, Training Loss: 8.3192, Validation Loss: 8.3590
Best model saved at epoch 2 with validation loss: 8.3590
Epoch 3/10, Training Loss: 6.9464, Validation Loss: 7.6235
Best model saved at epoch 3 with validation loss: 7.6235
Epoch 4/10, Training Loss: 6.2921, Validation Loss: 6.9456
Best model saved at epoch 4 with validation loss: 6.9456
Epoch 5/10, Training Loss: 5.3403, Validation Loss: 6.3800
Best model saved at epoch 5 with validation loss: 6.3800
Epoch 6/10, Training Loss: 4.7640, Validation Loss: 5.9190
Best model saved at epoch 6 with validation loss: 5.9190
Epoch 7/10, Training Loss: 3.3056, Validation Loss: 5.6187
Best model saved at epoch 7 with validation loss: 5.6187
Epoch 8/10, Training Loss: 2.7980, Validation Loss: 5.7395
Epoch 9/10, Training Loss: 2.8324, Validation Loss: 5.1642
Best model saved at epoch 9 with validation loss: 5.1642
Epoch 10/10,

Causal Language Modeling for Next Word Prediction [with question masking]

In [5]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
import torch.nn.functional as F
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare 5 QA samples for training and validation
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = tokenizer.eos_token_id  # Ignore question and padding tokens

    inputs["labels"] = labels
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Convert to PyTorch DataLoader
batch_size = 2

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["labels"] for item in batch])
    return input_ids, attention_mask, labels

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Cross-entropy loss function ignoring padding tokens
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model based on validation loss
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./gpt2-qa-best-loss-cml"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training and evaluation functions
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # Save best model based on validation loss
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10)

# Load the best fine-tuned model for inference
model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Training Loss: 4.4203, Validation Loss: 1.5726
Best model saved at epoch 1 with validation loss: 1.5726
Epoch 2/10, Training Loss: 0.9883, Validation Loss: 0.9799
Best model saved at epoch 2 with validation loss: 0.9799
Epoch 3/10, Training Loss: 0.7012, Validation Loss: 0.6840
Best model saved at epoch 3 with validation loss: 0.6840
Epoch 4/10, Training Loss: 0.4719, Validation Loss: 0.4885
Best model saved at epoch 4 with validation loss: 0.4885
Epoch 5/10, Training Loss: 0.2782, Validation Loss: 0.4139
Best model saved at epoch 5 with validation loss: 0.4139
Epoch 6/10, Training Loss: 0.2091, Validation Loss: 0.3706
Best model saved at epoch 6 with validation loss: 0.3706
Epoch 7/10, Training Loss: 0.1939, Validation Loss: 0.3273
Best model saved at epoch 7 with validation loss: 0.3273
Epoch 8/10, Training Loss: 0.1228, Validation Loss: 0.2700
Best model saved at epoch 8 with validation loss: 0.2700
Epoch 9/10, Training Loss: 0.1057, Validation Loss: 0.2148
Best model sa

Causal Language Modeling for Next Word Prediction [without question masking]

In [10]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
import torch.nn.functional as F
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 42
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare 5 QA samples for training and validation
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels and mask padding tokens efficiently using list comprehension
    labels = [label if label != tokenizer.pad_token_id else -100 for label in inputs["input_ids"]]

    inputs["labels"] = labels
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Convert to PyTorch DataLoader
batch_size = 2

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["labels"] for item in batch])
    return input_ids, attention_mask, labels

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Cross-entropy loss function ignoring padding tokens
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model based on validation loss
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./gpt2-qa-best-loss-cml-nomask"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training and evaluation functions
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # Save best model based on validation loss
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=5)

# Load the best fine-tuned model for inference
model_name = "./gpt2-qa-best-loss-cml-nomask"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Training Loss: 2.6022, Validation Loss: 1.8034
Best model saved at epoch 1 with validation loss: 1.8034
Epoch 2/5, Training Loss: 1.4668, Validation Loss: 1.4203
Best model saved at epoch 2 with validation loss: 1.4203
Epoch 3/5, Training Loss: 0.9887, Validation Loss: 1.2074
Best model saved at epoch 3 with validation loss: 1.2074
Epoch 4/5, Training Loss: 0.6512, Validation Loss: 1.1086
Best model saved at epoch 4 with validation loss: 1.1086
Epoch 5/5, Training Loss: 0.4697, Validation Loss: 1.0447
Best model saved at epoch 5 with validation loss: 1.0447
Q: What is the capital of France?
A: 


#Inference: Greedy Search

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def greedy_generate_answer(question, max_length=50):
    """
    Generate an answer using greedy search.

    Args:
        question (str): The input question.
        max_length (int): Maximum length of the generated sequence.

    Returns:
        str: The generated answer.
    """
    model.eval()
    device = model.device

    # Prepare input with the instruction-based format
    input_text = f"Question: {question} Answer:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Initialize generated token IDs with input
    generated_ids = input_ids.clone()

    with torch.no_grad():
        for _ in range(max_length):
            # Get the model's output logits
            outputs = model(input_ids=generated_ids)
            logits = outputs.logits

            # Select the token with the highest probability (greedy search)
            next_token_id = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)

            # Append the new token to the sequence
            generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

            # Stop generating if the EOS token is reached
            if next_token_id.item() == tokenizer.eos_token_id:
                break

    # Decode the generated token IDs to text
    answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Load the fine-tuned model and tokenizer
# model_name = "./gpt2-qa-finetuned"  # Change to your model path
# model_name = "./gpt2-qa-best"
model_name = "./gpt2-qa-best-loss"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# Ensure padding token is set
tokenizer.pad_token = tokenizer.eos_token

# Example usage
question = "What is the capital of France?"
answer = greedy_generate_answer(question)
print(f"Q: {question}\nA: {answer}")


Q: What is the capital of France?
A: Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris.


#Inference: Beam Search

In [ ]:
import torch
import torch.nn.functional as F
import time

def generate_answer(question, model, tokenizer, max_length=50, num_beams=5, device="cuda"):
    """
    Generate an answer using beam search for the fine-tuned GPT-2 model.

    Args:
        question (str): The input question.
        model (GPT2LMHeadModel): The fine-tuned model.
        tokenizer (GPT2Tokenizer): The tokenizer.
        max_length (int): Maximum length of the generated text.
        num_beams (int): Number of beams for beam search.
        device (str): The device to run the model on ('cuda' or 'cpu').

    Returns:
        str: The generated answer.
    """
    model.to(device)
    model.eval()

    # Prepare input text
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True)

    # Extract input_ids and attention_mask
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Initialize beams
    beams = [(input_ids, attention_mask, 0.0)]  # List of (sequence, attention_mask, score)
    completed_sequences = []

    # Beam search loop
    for _ in range(max_length):
        new_beams = []

        for seq, mask, score in beams:
            # Stop expanding if sequence ends with EOS token
            if seq[0, -1] == tokenizer.eos_token_id:
                completed_sequences.append((seq, score))  # Append sequence and score
                continue

            # Forward pass
            with torch.no_grad():
                outputs = model(seq, attention_mask=mask)
                logits = outputs.logits[:, -1, :]  # Logits of the last token
                probs = F.log_softmax(logits, dim=-1)  # Convert to log probabilities

            # Get top-k tokens and their log probabilities
            top_k_probs, top_k_tokens = torch.topk(probs, num_beams, dim=-1)

            # Expand each beam
            for prob, token in zip(top_k_probs[0], top_k_tokens[0]):
                new_seq = torch.cat([seq, token.unsqueeze(0).unsqueeze(0)], dim=1)  # Append token
                new_mask = torch.cat([mask, torch.ones((1, 1), device=device)], dim=1)  # Update attention mask
                new_score = score + prob.item()  # Accumulate log probability as float
                new_beams.append((new_seq, new_mask, new_score))

        # Sort new beams by score and keep top-k
        new_beams = sorted(new_beams, key=lambda x: x[2], reverse=True)[:num_beams]
        beams = new_beams

        # Break if all beams end with EOS token
        if all(seq[0, -1] == tokenizer.eos_token_id for seq, _, _ in beams):
            break

    # Add remaining beams to completed sequences
    completed_sequences.extend([(seq, float(score)) for seq, _, score in beams])

    # Select the sequence with the highest score
    best_sequence, _ = max(completed_sequences, key=lambda x: x[1])

    # Decode the tokens to text
    answer = tokenizer.decode(best_sequence[0], skip_special_tokens=True)
    return answer


# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./gpt2-qa-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-qa-finetuned")

# Test the generate_answer function
question = "What is the capital of France?"
device = "cuda" if torch.cuda.is_available() else "cpu"
st = time.time()
answer = generate_answer(question, model, tokenizer, max_length=50, num_beams=5, device=device)
en = time.time()
print('total time:', en-st)
print(answer)

total time: 4.90141487121582
Question: What is the capital of France?
Answer:. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris
